# MONO - classify comments

In [1]:
# Get paths set either from environment variables or if not set use some default values
import os

if 'ACORA_HOME_PATH' in os.environ:
    acora_home_path = os.environ['ACORA_HOME_PATH']
else:
    acora_home_path = "../../acora"

if 'ACORA_DATA_PATH' in os.environ:
    data_path = os.environ['ACORA_DATA_PATH']
else:
    data_path = "./data"
    
if 'BERT_PRETRAIN_MODELS_PATH' in os.environ:
    berts_pretrain_path = os.environ['BERT_PRETRAIN_MODELS_PATH']
else:
    berts_pretrain_path = "../bert"

acora_home_path, data_path, berts_pretrain_path

('C:\\Users\\user\\Research\\acora-pure',
 'E:\\GoogleDrive\\acora-data',
 'D:\\Research\\Datasets\\BERT')

In [2]:
random_seed = 102329

## Imports

In [3]:
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import json

import pandas as pd
import numpy as np

import json

from scipy import stats

from sklearn.model_selection import train_test_split

from collections import Counter

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef as mcc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import math


import warnings  
with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)

    import tensorflow as tf

    if tf.__version__.startswith("1."):
        os.environ['TF_KERAS'] = '0'
        from tensorflow import ConfigProto, Session, set_random_seed
        import keras
        from keras.backend.tensorflow_backend import set_session
        from keras.backend.tensorflow_backend import clear_session
        from keras.backend.tensorflow_backend import get_session
    else:
        os.environ['TF_KERAS'] = '1'
        from tensorflow.compat.v1 import ConfigProto, Session, set_random_seed
        import tensorflow.compat.v1.keras as keras
        tf.get_logger().setLevel('INFO')
         
    from tensorflow.python.client import device_lib


    from keras_bert import Tokenizer, load_trained_model_from_checkpoint
    from keras_bert.layers.extract import Extract

    from keras_radam import RAdam

from acora.vocab import BERTVocab
from acora.comments import default_subject_columns, \
    load_comments_files, CommentPurposeTransformer, CommentSubjectTransformer, \
    plot_purpose_confusion_matrix, plot_subjects_confusion_matrix, \
    report_comment_predictions_accuracy, default_purpose_labels, save_comment_predictions_accuracy

## Load data

In [4]:
#bert_name = 'uncased_L-8_H-512_A-8'
bert_name = 'multi_cased_L-12_H-768_A-12'

config_path = os.path.join(berts_pretrain_path, bert_name, 'bert_config.json')
checkpoint_path = os.path.join(berts_pretrain_path, bert_name, 'bert_model.ckpt')
vocab_path = os.path.join(berts_pretrain_path, bert_name, 'vocab.txt')
with open(config_path, "r", encoding='utf', errors='ignore') as json_file:
    bert_config = json.load(json_file)

In [5]:
not_use_gpu = False

In [6]:
gpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
if not not_use_gpu and len(gpus) == 0:
    logger.error("You don't have a GPU available on your system, it can affect the performance...")

for gpu_entry in device_lib.list_local_devices():
    if hasattr(gpu_entry, 'physical_device_desc'):
        print(f"{gpu_entry.name}: {gpu_entry.physical_device_desc}, {gpu_entry.memory_limit}")

/device:CPU:0: , 268435456
/device:XLA_CPU:0: device: XLA_CPU device, 17179869184
/device:GPU:0: device: 0, name: NVIDIA TITAN Xp COLLECTORS EDITION, pci bus id: 0000:05:00.0, compute capability: 6.1, 10857253312
/device:XLA_GPU:0: device: XLA_GPU device, 17179869184


In [7]:
vocab = BERTVocab.load_from_file(vocab_path)

In [8]:
tokenizer = Tokenizer(vocab.token_dict)

In [10]:
sep = "$"

line_column = "line_contents"
message_column = "message"
purpose_column = "purpose"
subject_columns = default_subject_columns

cols = [line_column, message_column, purpose_column] + subject_columns

training_data_paths = [
    os.path.join(data_path, "mono", "mono-all.xlsx")
]
reviews_all_df = load_comments_files(training_data_paths, cols, sep)

Loading data from E:\GoogleDrive\acora-data\mono\mono-all.xlsx
Loaded 172 rows and 15 cols...


Check if there are any duplicated comments

In [11]:
print(f"Number of comments = {reviews_all_df.shape[0]}")
print(f"Number of unique comments = {reviews_all_df[message_column].unique().shape[0]}")

print(f"Number of duplicated lines and comments = {pd.concat(g for _, g in reviews_all_df.groupby([line_column, message_column]) if len(g) > 1).shape[0]}")


Number of comments = 172
Number of unique comments = 111
Number of duplicated lines and comments = 4


The duplicated comments are mostly no exact duplicates but rather the same comments made for similar cases. 

We are going to study the optimistic scenario since it correspond to a real-life use case.

In [12]:
duplicates = pd.concat(g for _, g in reviews_all_df.groupby(message_column) if len(g) > 1)
unique = pd.concat(g for _, g in reviews_all_df.groupby(message_column) if len(g) == 1)
reviews_all_optimistic_df = pd.concat([duplicates, unique])

In [13]:
reviews_all_optimistic_df.shape

(172, 15)

In [14]:
seq_len = 128
comments_lengths = [len(tokenizer.encode(str(text))[0]) for text in reviews_all_df[message_column].tolist()]
print("Message lengths distribution: 90% is {:.0f}, 95% is {:.0f}, 98% is {:.0f}, 99% is {:.0f}, and 100% is {}".format(
        *np.percentile(comments_lengths, [90, 95, 98, 99, 100])))
print(f"Your selected sequence length corresponds to {stats.percentileofscore(comments_lengths, seq_len):.2f} percentile in the training dataset.")

Message lengths distribution: 90% is 61, 95% is 89, 98% is 155, 99% is 164, and 100% is 164.0
Your selected sequence length corresponds to 95.35 percentile in the training dataset.


In [15]:
tokenized_all_optimistic_messages = [tokenizer.encode(str(text), max_len=seq_len)[0] for text in reviews_all_optimistic_df[message_column].tolist()] 
x_all_optimistic = [np.array(tokenized_all_optimistic_messages), np.zeros_like(tokenized_all_optimistic_messages)]


In [16]:
subject_transformer_optimistic = CommentSubjectTransformer(reviews_all_optimistic_df, subject_columns)
y_all_optimistic_subject = subject_transformer_optimistic.encode_one_hot_all_subjects()

## Train model

In [17]:
model = None

def get_model(seq_len, subject_columns, layer_num, config_path, checkpoint_path, lr=2e-5, 
              not_use_gpu = not_use_gpu):
    
    global model

    try:
        del model 
    except:
        print("Unable to delete the model")
    
    if tf.__version__.startswith("1."):
        sess = get_session()
        clear_session()
        sess.close()
        gpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
        if not not_use_gpu and len(gpus) == 0:
            logger.error("You don't have a GPU available on your system, it can affect the performance...")

        config = ConfigProto( device_count = {'GPU': 0 if not_use_gpu else len(gpus)})
        config.gpu_options.per_process_gpu_memory_fraction = 1
        config.gpu_options.visible_device_list = "0"
        sess = Session(config=config)
        keras.backend.set_session(sess)
    else:
        sess = tf.compat.v1.keras.backend.get_session()
        tf.compat.v1.keras.backend.clear_session()
        sess.close()
        gpus = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
        if not not_use_gpu and len(gpus) == 0:
            logger.error("You don't have a GPU available on your system, it can affect the performance...")

        config = ConfigProto( device_count = {'GPU': 0 if not_use_gpu else len(gpus)})
        config.gpu_options.per_process_gpu_memory_fraction = 1
        config.gpu_options.visible_device_list = "0"
        sess = Session(config=config)
        tf.compat.v1.keras.backend.set_session(sess)
    
    
    model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=seq_len
    )
    
    inputs = model.inputs[:2]
    #dense = model.get_layer(f'Encoder-{layer_num}-FeedForward-Norm').output
    #dense = Extract(index=0, name="Extract")(dense)
    dense = model.get_layer('NSP-Dense').output
    dense = keras.layers.Dropout(0.1)(dense)


    losses = dict()
    loss_weights = dict()
    outputs = []
   
    for i, subject_class in enumerate(subject_columns):
        outputs.append(keras.layers.Dense(units=1, activation='sigmoid', name=f"{subject_class}_output")(dense))
        losses[f"{subject_class}_output"] = "binary_crossentropy"
        loss_weights[f"{subject_class}_output"] = 1.0

    model = keras.models.Model(inputs, outputs)
    
    model.compile(
        RAdam(learning_rate=lr,beta_1=0.9, beta_2=0.999,warmup_proportion=0.1),
        loss=losses, 
        loss_weights=loss_weights,
        metrics=['accuracy'],
    )
    
    return model


In [18]:
y_all_optimistic = dict()
for i, subject_class in enumerate(subject_columns):
    y_all_optimistic[f"{subject_class}_output"] = subject_transformer_optimistic.encode_binary_single_subject(subject_class).values


In [19]:
subject_class_weights_optimistic = subject_transformer_optimistic.class_weights()
print(f"Calculated subject weights: {subject_class_weights_optimistic}")

class_weights_all_optimistic = dict()
for i, subject_class in enumerate(subject_columns):
    class_weights_all_optimistic[f"{subject_class}_output"] = subject_class_weights_optimistic[subject_class]


Calculated subject weights: {'code_design': array([ 0.50887574, 28.66666667]), 'code_style': array([0.57333333, 3.90909091]), 'code_naming': array([ 0.51497006, 17.2       ]), 'code_logic': array([0.86868687, 1.17808219]), 'code_io': array([ 0.50292398, 86.        ]), 'code_data': array([0.81904762, 1.28358209]), 'code_doc': array([1.]), 'code_api': array([0.55128205, 5.375     ]), 'compatibility': array([ 0.51497006, 17.2       ]), 'rule_def': array([ 0.50588235, 43.        ]), 'config_commit_patch_review': array([1.]), 'config_building_installing': array([ 0.50588235, 43.        ])}


In [20]:
def validate_model(x_all, y_all, random_seed=0, runs=10, n_splits=10, batch_size = 24, 
                   epochs = 1, layers_num = 12, figsize=(10,20), cmap='Greens',
                   seq_len=128, subject_columns=subject_columns, config_path=config_path, 
                   checkpoint_path=checkpoint_path, variant="", lr=2e-5):

    np.random.seed(random_seed)
    set_random_seed(random_seed)

    X = x_all
    y = y_all

    results = dict()
    
    y_preds_dfs = []
    y_trues_dfs = []

    for run in range(runs):
        kfold = KFold(n_splits=n_splits, shuffle=True)
        print(f'Run {run+1} / {runs}')
        print('- Fold: ', end=" ")
        fold = 1
        subject_preds_dfs = []
        y_true_dfs = []
        for train_index, val_index in kfold.split(X[0]): 
            print(f'{fold}', end=' ')

            y_train = {k:v[train_index] for k,v in y.items()}
            X_train = [X[0][train_index,:], X[1][train_index,:]]


            model = get_model(seq_len, subject_columns, layers_num, config_path, checkpoint_path, lr)

            history = model.fit(
                X_train,
                y_train,
                epochs=epochs,
                batch_size=batch_size,
                verbose=1,
                shuffle=True,
            )
            print("Predicting...")
            y_val = {k:v[val_index] for k,v in y.items()}
            X_val = [X[0][val_index,:], X[1][val_index,:]]

            y_pred_subject = model.predict(X_val) 

            y_pred_subject = np.array(y_pred_subject).reshape(len(y_pred_subject),len(y_pred_subject[0])).transpose()
            subject_all_preds = []
            for preds in y_pred_subject:
                subject_all_preds.append([1 if x > 0.5 else 0 for x in preds]) 
            subject_preds_df = pd.DataFrame(subject_all_preds, columns=subject_columns)
            subject_preds_dfs.append(subject_preds_df)
            y_true_dfs.append(pd.DataFrame(y_val))

            fold += 1
        print("")

        subject_preds_df = pd.concat(subject_preds_dfs)
        y_true_df = pd.concat(y_true_dfs)
        y_true_df.columns = [str(col).replace("_output", "") for col in y_true_df.columns]

        results[run] = dict()
        for subject in subject_columns:
            subject_acc = accuracy_score(y_true_df[subject], subject_preds_df[subject])
            subject_f1 = f1_score(y_true_df[subject], subject_preds_df[subject], average="macro")
            subject_precision = precision_score(y_true_df[subject], subject_preds_df[subject], average="macro")
            subject_recall = recall_score(y_true_df[subject], subject_preds_df[subject], average="macro")
            subject_f1_micro = f1_score(y_true_df[subject], subject_preds_df[subject], average="micro")
            subject_precision_micro = precision_score(y_true_df[subject], subject_preds_df[subject], average="micro")
            subject_recall_micro = recall_score(y_true_df[subject], subject_preds_df[subject], average="micro")
            subject_f1_binary = f1_score(y_true_df[subject], subject_preds_df[subject], average="binary")
            subject_precision_binary = precision_score(y_true_df[subject], subject_preds_df[subject], average="binary")
            subject_recall_binary = recall_score(y_true_df[subject], subject_preds_df[subject], average="binary")
            subject_mcc = mcc_score(y_true_df[subject], subject_preds_df[subject])
            try:
                subject_roc_auc_score = roc_auc_score(y_true_df[subject], subject_preds_df[subject])
                print(f"Subject {subject} AUC = {subject_roc_auc_score:.2}")
            except:
                subject_roc_auc_score = None
            print(f"Subject {subject} Accuracy = {subject_acc:.2f}")
            print(f"Subject {subject} Precision (macro) = {subject_precision:.2}")
            print(f"Subject {subject} Recall (macro) = {subject_recall:.2}")
            print(f"Subject {subject} F1-score (macro) = {subject_f1:.2}")
            print(f"Subject {subject} Precision (micro) = {subject_precision_micro:.2}")
            print(f"Subject {subject} Recall (micro) = {subject_recall_micro:.2}")
            print(f"Subject {subject} F1-score (micro) = {subject_f1_micro:.2}")
            print(f"Subject {subject} Precision (binary) = {subject_precision_binary:.2}")
            print(f"Subject {subject} Recall (binary) = {subject_recall_binary:.2}")
            print(f"Subject {subject} F1-score (binary) = {subject_f1_binary:.2}")
            print(f"Subject {subject} MCC = {subject_mcc:.2}")
            

            results[run][subject] = dict()
            results[run][subject]['acc'] = subject_acc
            results[run][subject]['rec_macro']= subject_recall
            results[run][subject]['prec_macro'] = subject_precision
            results[run][subject]['fscore_macro'] = subject_f1
            results[run][subject]['rec_micro']= subject_recall_micro
            results[run][subject]['prec_micro'] = subject_precision_micro
            results[run][subject]['fscore_micro'] = subject_f1_micro
            results[run][subject]['rec_binary']= subject_recall_binary
            results[run][subject]['prec_binary'] = subject_precision_binary
            results[run][subject]['fscore_binary'] = subject_f1_binary
            results[run][subject]['mcc'] = subject_mcc
            results[run][subject]['auc'] = subject_roc_auc_score

        subject_cm_path=f"./output/mono-{variant}-run-{run}-e-{epochs}-subject_cm.pdf"
        print("Preparing confusion matrix for the comment subjects.")
        
        y_trues_dfs.append(y_true_df[subject_columns])
        y_preds_dfs.append(subject_preds_df[subject_columns])
        cf_matrix_all_subject = multilabel_confusion_matrix(y_true_df[subject_columns].values,
                                                            subject_preds_df[subject_columns].values, samplewise=False)

        fig = plt.figure(figsize=figsize)
        cols = math.ceil(float(len(subject_columns)) / 2.0)
        #print(cols)
        gs = gridspec.GridSpec(cols, 2, height_ratios=[1]*cols)
        gs.update(hspace=0.4, wspace=0.5)

        for i, cf in enumerate(cf_matrix_all_subject):

            row = i // 2
            col = i % 2
            #print(row, col)
            ax = plt.subplot(gs[row, col])

            cmn = cf.astype('float') / cf.sum(axis=1)[:, np.newaxis]
            perc_labs = ["{0:.1%}".format(value) for value in cmn.flatten()]

            group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]

            box_labels = [f"{v1}{v2}".strip() for v1, v2 in zip(group_counts,perc_labs)]
            box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

            sns.heatmap(cmn, 
                        annot=box_labels, 
                        fmt='', 
                        annot_kws={"fontsize":12},
                        xticklabels=("False", "True"), 
                        yticklabels=("False", "True"),
                    cmap=cmap,
                    linecolor='lightgray', linewidths=0.5,
                    square=True,
                    cbar=False,
                    vmin=0, vmax=1)
            ax.set_title(subject_columns[i])
            plt.ylabel('True label')
            plt.xlabel('Predicted label')

        plt.tight_layout()
        plt.savefig(subject_cm_path)
        print(f"Confusion matrix for the comment subject saved to {subject_cm_path}.")
        plt.close()
        
        
        
    subject_cm_path=f"./output/mono-{variant}-e-{epochs}-all-subject_cm.pdf"
    print("Preparing confusion matrix for the comment subjects.")

    y_true_df = pd.concat(y_trues_dfs)
    y_preds_df = pd.concat(y_preds_dfs)
    cf_matrix_all_subject = multilabel_confusion_matrix(y_true_df[subject_columns].values,
                                                        y_preds_df[subject_columns].values, samplewise=False)

    fig = plt.figure(figsize=figsize)
    cols = math.ceil(float(len(subject_columns)) / 2.0)
    #print(cols)
    gs = gridspec.GridSpec(cols, 2, height_ratios=[1]*cols)
    gs.update(hspace=0.4, wspace=0.5)

    for i, cf in enumerate(cf_matrix_all_subject):

        row = i // 2
        col = i % 2
        #print(row, col)
        ax = plt.subplot(gs[row, col])

        cmn = cf.astype('float') / cf.sum(axis=1)[:, np.newaxis]
        perc_labs = ["{0:.1%}".format(value) for value in cmn.flatten()]

        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]

        box_labels = [f"{v1}{v2}".strip() for v1, v2 in zip(group_counts,perc_labs)]
        box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

        sns.heatmap(cmn, 
                    annot=box_labels, 
                    fmt='', 
                    annot_kws={"fontsize":12},
                    xticklabels=("False", "True"), 
                    yticklabels=("False", "True"),
                cmap=cmap,
                linecolor='lightgray', linewidths=0.5,
                square=True,
                cbar=False,
                vmin=0, vmax=1)
        ax.set_title(subject_columns[i])
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

    plt.tight_layout()
    plt.savefig(subject_cm_path)
    print(f"Confusion matrix for the comment subject saved to {subject_cm_path}.")
    plt.close()
        
    return results




In [26]:
results_optimistic = validate_model(x_all_optimistic, 
                                    y_all_optimistic, 
                                    runs=10, n_splits=10,
                                    epochs = 15,
                                    lr=1e-4,
                                    random_seed=random_seed,
                                    variant="optimistic")

Run 1 / 10
- Fold:  1 WARNING:tensorflow:AutoGraph could not transform <bound method TokenEmbedding.call of <keras_bert.layers.embedding.TokenEmbedding object at 0x0000029F047ADE48>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: modul

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7/7 [==============================] - 8s 1s/step - loss

Epoch 8/15
7/7 [==============================] - 7s 964ms/step - loss: 2.1818 - code_design_output_loss: 0.0914 - code_style_output_loss: 0.2677 - code_naming_output_loss: 0.1138 - code_logic_output_loss: 0.4879 - code_io_output_loss: 0.0653 - code_data_output_loss: 0.5091 - code_doc_output_loss: 0.0394 - code_api_output_loss: 0.2819 - compatibility_output_loss: 0.1319 - rule_def_output_loss: 0.0820 - config_commit_patch_review_output_loss: 0.0380 - config_building_installing_output_loss: 0.0735 - code_design_output_accuracy: 0.9870 - code_style_output_accuracy: 0.8896 - code_naming_output_accuracy: 0.9740 - code_logic_output_accuracy: 0.8117 - code_io_output_accuracy: 0.9935 - code_data_output_accuracy: 0.7987 - code_doc_output_accuracy: 1.0000 - code_api_output_accuracy: 0.9091 - compatibility_output_accuracy: 0.9740 - rule_def_output_accuracy: 0.9870 - config_commit_patch_review_output_accuracy: 1.0000 - config_building_installing_output_accuracy: 0.9870
Epoch 9/15
7/7 [===========

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2 Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7/7 [==============================] - 7s 982ms/step -

Epoch 8/15
7/7 [==============================] - 7s 964ms/step - loss: 2.2338 - code_design_output_loss: 0.1078 - code_style_output_loss: 0.2986 - code_naming_output_loss: 0.1409 - code_logic_output_loss: 0.4595 - code_io_output_loss: 0.0684 - code_data_output_loss: 0.4721 - code_doc_output_loss: 0.0394 - code_api_output_loss: 0.2742 - compatibility_output_loss: 0.1517 - rule_def_output_loss: 0.0881 - config_commit_patch_review_output_loss: 0.0468 - config_building_installing_output_loss: 0.0863 - code_design_output_accuracy: 0.9805 - code_style_output_accuracy: 0.8701 - code_naming_output_accuracy: 0.9675 - code_logic_output_accuracy: 0.8182 - code_io_output_accuracy: 0.9935 - code_data_output_accuracy: 0.7987 - code_doc_output_accuracy: 1.0000 - code_api_output_accuracy: 0.9026 - compatibility_output_accuracy: 0.9675 - rule_def_output_accuracy: 0.9870 - config_commit_patch_review_output_accuracy: 1.0000 - config_building_installing_output_accuracy: 0.9870
Epoch 9/15
7/7 [===========

...

Subject code_api AUC = 0.72
Subject code_api Accuracy = 0.94
Subject code_api Precision (macro) = 0.91
Subject code_api Recall (macro) = 0.72
Subject code_api F1-score (macro) = 0.78
Subject code_api Precision (micro) = 0.94
Subject code_api Recall (micro) = 0.94
Subject code_api F1-score (micro) = 0.94
Subject code_api Precision (binary) = 0.88
Subject code_api Recall (binary) = 0.44
Subject code_api F1-score (binary) = 0.58
Subject code_api MCC = 0.59
Subject compatibility AUC = 0.7
Subject compatibility Accuracy = 0.98
Subject compatibility Precision (macro) = 0.99
Subject compatibility Recall (macro) = 0.7
Subject compatibility F1-score (macro) = 0.78
Subject compatibility Precision (micro) = 0.98
Subject compatibility Recall (micro) = 0.98
Subject compatibility F1-score (micro) = 0.98
Subject compatibility Precision (binary) = 1.0
Subject compatibility Recall (binary) = 0.4
Subject compatibility F1-score (binary) = 0.57
Subject compatibility MCC = 0.63
Subject rule_def AUC = 0.5
S

C:\Users\user\Anaconda3\envs\ml2\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\Anaconda3\envs\ml2\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\Anaconda3\envs\ml2\lib\site-packages\sklearn\metrics\_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\user\Anaconda3\envs\ml2\lib\site-packages\sklearn\metrics\_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted 

Confusion matrix for the comment subject saved to ./output/mono-pesimistic-run-9-e-15-subject_cm.pdf.
Preparing confusion matrix for the comment subjects.


C:\Users\user\Anaconda3\envs\ml2\lib\site-packages\ipykernel_launcher.py:179: RuntimeWarning: invalid value encountered in true_divide


Confusion matrix for the comment subject saved to ./output/mono-pesimistic-e-15-all-subject_cm.pdf.


In [27]:
with open('./output/res-mono-optimistic-e15.json', 'w') as fp:
    json.dump(results_optimistic, fp, indent=4)

In [28]:
results_optimistic_df = pd.concat([pd.DataFrame(v) for k, v in results_optimistic.items()], axis=0) 
results_optimistic_df.to_excel('./output/res-mono-optimistic-e15.xlsx')

In [29]:
results_df = results_optimistic_df
reviews_df = reviews_all_optimistic_df

counts_df = reviews_df[subject_columns].sum()
counts_df.name = "n"

perc_counts_df = reviews_df[subject_columns].sum() / reviews_df.shape[0]
perc_counts_df.name = "perc_count"

summary_dfs = [counts_df, perc_counts_df]
for metric in results_df.index.unique().tolist():
    metric_df = pd.concat([results_df[results_df.index == metric].mean(), results_df[results_df.index == metric].std()], axis=1)
    metric_df.columns = [metric, f"{metric}_std"]
    print(metric_df.shape)
    summary_dfs.append(metric_df)
    
summary_df = pd.concat(summary_dfs, axis=1)
summary_df.to_excel('./output/summary-mono-optimistic-e15.xlsx')

(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)


In [106]:
summary_df.shape

(12, 242)